In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [ ]:
#df_trans = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',low_memory=False)
#df_art = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
df_cus = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')


#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

In [ ]:
df_trans.head()

In [ ]:
df_cus.head()

In [ ]:
print(df_cus['customer_id'].nunique())
print(df_trans['customer_id'].nunique())

In [ ]:
df_trans.t_dat.max()

### Task 1 - Use age, R, F, M values for basic customer clustering/ Can include FN, Active,club_member_status, fashion_news_frequency
### EDA of FN, Active,club_member_status, fashion_news_frequency - Do we need to take them into account
### Produce a list of articles bought by each customer 
### Create the User product matrix
### Use ALS

## Calculate the RFM values for each customer from transcation data

In [ ]:
df_trans['Frequency']=df_trans.groupby('customer_id')['article_id'].transform('count')
df_trans['Monetary']=df_trans.groupby('customer_id')['price'].transform('sum')

In [ ]:
order_dt = df_trans.groupby('customer_id')['t_dat'].max().rename('max_order_dt').reset_index()
df_trans_2=pd.merge(df_trans,order_dt,how='left',on='customer_id')

In [ ]:
df_RFM = df_trans_2[['customer_id','Frequency','Monetary','max_order_dt']].drop_duplicates().reset_index()
df_RFM['Recency']= (pd.to_datetime(df_RFM['max_order_dt'].max())-pd.to_datetime(df_RFM['max_order_dt'])).dt.days


In [ ]:
df_RFM.dtypes

In [ ]:
print(df_RFM.shape[0])
print(df_trans['customer_id'].nunique())

#### EDA for FN, Active, club member and fashion_news
#### FN is if a customer get Fashion News newsletter, Active is if the customer is active for communication, sales channel id, 2 is online and 1 store.

In [ ]:
df_cus.head()

In [ ]:
print((df_cus.isna().sum()/df_cus.shape[0])*100)
print(df_cus.shape[0])

In [ ]:
df_cus['FN'].fillna(0,inplace=True)
df_cus['Active'].fillna(0,inplace=True)
df_cus['club_member_status'].fillna('NONE',inplace=True)
df_cus['fashion_news_frequency'].fillna('NONE',inplace=True)
df_cus.loc[df_cus['fashion_news_frequency']=='None','fashion_news_frequency']='NONE'


imp = SimpleImputer()
df_cus['age']= imp.fit_transform(df_cus['age'].values.reshape(-1,1))


In [ ]:
for col in df_cus.columns[1:-1] :
    print('-'*50)
    
    print(col.upper())
    
    print(df_cus[col].value_counts())
    
   
    

#### 35% of the customers are open to fashion newsletter
#### 34% of the customers are active for communication
#### Create dummies for these variables and use in clustering

In [ ]:
df_cus['fashion_news_frequency'].value_counts()

In [ ]:
df_cus['Active'].value_counts()

In [ ]:
df_cus['club_member_status'].value_counts()/df_cus.shape[0]

In [ ]:
df_cus['FN'].value_counts()

### Subset the data for FN =1
### Check the distribution of fashion news_frequency, active 
### 92% of the customers have club_member_status as Active.We should not use this for clustering as most of the customers exhibit same behaviour

In [ ]:
QC= df_cus[df_cus['FN']==1]
QC['Active'].fillna(0,inplace=True)
QC.head()
QC.shape

In [ ]:
cols = ['Active','fashion_news_frequency']
for col in cols:
    print(col.upper())
    print(QC[col].value_counts())
    print('missing values are',round((QC[col].isna().sum()/QC.shape[0])*100,2),'%')

##### FN is if a customer get Fashion News newsletter, Active is if the customer is active for communication, sales channel id, 2 is online and 1 store.
##### Answered by competition host

In [ ]:
print(df_trans['sales_channel_id'].value_counts())
print('no of unique customers are', df_trans['customer_id'].nunique())
print('no of unique products:',df_trans['article_id'].nunique())
#print('We have',(pd.to_datetime(df_trans['t_dat']).max()-pd.to_datetime(df_trans['t_dat']).min()).days,'of data ranging from',df_trans['t_dat'].min(),'to',df_trans['t_dat'].max())

#data_range = pd.to_datetime(df_trans['t_dat']).max()-pd.to_datetime(df_trans['t_dat']).min()).days
data_min = pd.to_datetime(df_trans['t_dat']).min()
data_max = pd.to_datetime(df_trans['t_dat']).max()

print(f"The data has {(data_max-data_min).days} days and ranges from {data_min} to {data_max}")
print("The data has %d days and ranges from %s to %s"%((data_max-data_min).days,data_min,data_max))


## Monthly Sales figure for customers (Time Series)
## 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [ ]:
plt.figure(figsize = (18,16)) 
df_trans.groupby('t_dat')['article_id'].count().plot(kind='line',xlabel='Date',ylabel="count of purchases", title="Count of Products purchased with time")

In [ ]:
df= pd.DataFrame(df_trans.groupby('sales_channel_id')['article_id'].count().reset_index())
plt.bar(df['sales_channel_id'],df['article_id'])
x_labels =['sales_chnl_1','sales_chnl_2']
plt.xticks(df['sales_channel_id'],x_labels)
#plt.ticklabel_format.ScalarFormatter(style='plain')
plt.xlabel('sales_channel_id')
plt.ylabel('count of articles purchased')
plt.margins(0.2)
plt.show()

In [ ]:
df_trans.columns

In [ ]:
df_trans.groupby(['customer_id',pd.to_datetime(df_trans['t_dat']).dt.year])['article_id'].count().sort_values(ascending=False).groupby('t_dat').sum().reset_index()

## Customers data

In [ ]:
import pandas as pd
df_cus = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")

#### "FN" is Fashion News and "Active" = 1 when the customer is active for communication

In [ ]:
df_cus.head(5)

In [ ]:
df_cus['club_member_status'].value_counts()

In [ ]:
df_cus['age'].hist()

In [ ]:
import pandas as pd
df_sku= pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')

In [ ]:
df_cus.head()

In [ ]:
print(df_sku['article_id'].nunique())
print(df_sku['product_code'].nunique())

In [ ]:
df_sku.nunique()